<script type="text/x-mathjax-config">
MathJax.Hub.Config({
  TeX: { equationNumbers: { autoNumber: "AMS" } }
});
</script>

## ガウス過程

#### 用語の整理
1. 動径基底関数
    $$
    \begin{align}
    \exp \left(- \frac{\left(x - \mu \right)^{2}}{\sigma^{2}} \right)
    \end{align}
    $$
    のような形の関数のこと。オイラーの公式で三角関数に分解できることを想像すると、名前に納得感があると思った。

1. 動径基底関数（radial basis function, RBF）回帰  
    線形回帰モデルの基底関数に、動径基底関数を用いたモデル。  
    $$
    \begin{align}
    y = \sum_{h=-H}^{H} w_{h} \exp \left(- \frac{\left(x - \mu \right)^{2}}{\sigma^{2}} \right)
    \end{align}
    $$
    データが低次元じゃないと計算できない（次元の呪い）。たとえば、1次元なら任意の曲線を表現するために$x$の値を細かくとれば良いが、2次元で任意の局面を表現するためには細かいメッシュを切る必要がある。

#### ガウス過程
任意次元の入力$x$に対する出力$y$の同時分布が多変量ガウス分布に従うもののこと。無限次元ガウス分布を、データのある次元だけを残して周辺化してしまったもの。正確な定義は書籍p67参照。

重み$\boldsymbol{w}$が、ガウス分布
$$
\begin{align}
\boldsymbol{w} \sim \mathcal{N} \left(\mathbf{0}, \lambda^{2} \boldsymbol{I} \right)
\end{align}
$$
から確率的に生成されると仮定して、その期待値をとって$w$を積分消去する。

$w$に関数を導入したことで、線形回帰モデルの任意の点（つまり無限点）の基底関数を表現可能になり、解析的に積分してしまうことで無限点を調べたことにしてしまうということ。無限点の基底関数を用意したのにも関わらず、この積分の計算（周辺化）によってデータ点がある次元だけで事後分布を表現できるのがすごいところ。

無限点とは何のことか、周辺化が何をしているのかのイメージをつかみたければp66の図3.5を参照のこと。ちゃんと理解したい人は再生核ヒルベルト空間について調べましょう。

$$
\begin{align}
\left(\begin{array}{c}{\widehat{y}_{1}} \\ {\widehat{y}_{2}} \\ {\vdots} \\ {\widehat{y}_{N}}\end{array}\right)&=\left(\begin{array}{cccc}{\phi_{0}\left(\boldsymbol{x}_{1}\right)} & {\phi_{1}\left(\boldsymbol{x}_{1}\right)} & {\cdots} & {\phi_{H}\left(\boldsymbol{x}_{1}\right)} \\ {\phi_{0}\left(\boldsymbol{x}_{2}\right)} & {\phi_{1}\left(\boldsymbol{x}_{2}\right)} & {\cdots} & {\phi_{H}\left(\boldsymbol{x}_{2}\right)} \\ {\vdots} & {} & {} & {\vdots} \\ {\phi_{0}\left(\boldsymbol{x}_{N}\right)} & {\phi_{1}\left(\boldsymbol{x}_{N}\right)} & {\cdots} & {\phi_{H}\left(\boldsymbol{x}_{N}\right)}\end{array}\right)\left(\begin{array}{c}{w_{0}} \\ {w_{1}} \\ {\vdots} \\ {\vdots} \\ {w_{H}}\end{array}\right) \\
\boldsymbol{\widehat y} &= \boldsymbol{\Phi w}
\end{align}
$$

とすると、
$$
\begin{align}
\mathbb{E}\left[\boldsymbol{y}\right] &= \mathbb{E}\left[\boldsymbol{\Phi w}\right] = \boldsymbol{\Phi} \mathbb{E}\left[\boldsymbol{w}\right] = \boldsymbol{0} \\
\Sigma &=\mathbb{E}\left[\boldsymbol{y y}^{\mathrm T}\right]-\mathbb{E}[\boldsymbol{y}] \mathbb{E}[\boldsymbol{y}]^{\mathrm T} \\
&= \mathbb{E}\left[(\boldsymbol{\Phi} \boldsymbol{w})(\boldsymbol{\Phi} \boldsymbol{w})^{\mathrm T}\right] \\
&= \boldsymbol{\Phi} \mathbb{E}\left[\boldsymbol{w} \boldsymbol{w}^{\mathrm T}\right] \boldsymbol{\Phi}^{\mathrm T} = \lambda^{2} \boldsymbol{\Phi} \Phi^{\mathrm T}
\end{align}
$$
より、
$$
\begin{align}
y \sim \mathcal{N} \left(\boldsymbol{0}, \lambda^{2}\boldsymbol{\Phi \Phi^{\mathrm T}} \right)
\end{align}
$$
となる。$\boldsymbol{w}$について知らなくてよい点で強力。

分散共分散行列（今回のような$A^{\mathrm T}A$の形の行列をグラム行列ともいう）$\boldsymbol{K} = \lambda^{2}\Phi\Phi^{\mathrm T}$の$n, n'$成分を見てみると、
$$
\begin{align}
\boldsymbol{K}_{n, n'} = \lambda^{2} \phi(\boldsymbol{x_{n}})^{\mathrm T}\phi(\boldsymbol{x_{n'}})
\end{align}
$$
となっており、基底関数の内積の定数倍となっている。$\boldsymbol{K}$は分散共分散行列なので、正則な対称行列であることに注意。


## カーネルトリック

上記の内積を、$\boldsymbol{x_{n}}$と$\boldsymbol{x_{n}}$のカーネル関数（kernel function）と呼び、以下のように定義する。
$$
\begin{align}
k(\boldsymbol{x}_{n}, \boldsymbol{x}_{n'}) = \phi(\boldsymbol{x_{n}})^{\mathrm T}\phi(\boldsymbol{x_{n'}}) 
\end{align}
$$
$\boldsymbol{w}$は積分消去されましたが、このままだと特徴ベクトル$\phi(\boldsymbol x)$が無限次元の場合表現できません。

そこで、$\phi(\cdot)$を直接表現することを避け、内積の結果の式だけを利用します（カーネルトリック）。

## 文献
1. [Kanagawa, Motonobu, et al. "Gaussian processes and kernel methods: A review on connections and equivalences." arXiv preprint arXiv:1807.02582 (2018).](https://arxiv.org/abs/1807.02582)  
    ベイズ統計の立場からみたカーネル法